In [1]:
import numpy as np
import sympy as sp
from IPython.display import HTML
import ipywidgets as widgets
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
import pandas as pd
import itertools

# This is commented out
#%matplotlib widget

# function to print latex
def renderListToLatex(e):
    latex_rendering = []

    for i in range(len(e)):
        latex_rendering.append("$" + sp.latex(e[i]) + "$ <br/> ")
    
    return(HTML("".join(latex_rendering[0:])))
    
    
def convert3DVectorToSQRTOfQuadrance(v):
    return(sp.N(sp.sqrt(sum([v.args[i].args[0]**2 for i in range(len(v.args))]))))



def convert3DVectorToComponents(v, scaleChoice):
    print(v)
    return(sp.parse_expr(str(v.args).replace('i', scaleChoice).replace("j", scaleChoice).replace("k", scaleChoice)))
    

<hr/>

<b>Aim</b>: Link equations relating to motion in 1 dimension

<hr/>


Let the following be unknown types

In [2]:
va, vi, v, v0, a, x, x0, t0, t1, m, s = sp.symbols('va, vi, v, v0, a, x, x0, t0, t1, m, s')

In [3]:
def createParticleMovementSummary(v = None, v0 = None, a = None, x = None, x0 = None, t1 = None, t0 = None, solutionsNeeded = []):
    #create summary
    print("###################### SUMMARY #######################")
    print("Initial Position \t (x0):\t", x0)
    print("Final Position \t\t (x):\t", x)
    print("Initial Velocity \t (v0):\t", v0)
    print("Final Velocity \t\t (v):\t", v)
    print("Acceleration \t\t (a):\t", a)
    print("Initial Time:\t\t (t0)\t", t0)
    print("Final Time:\t\t (t1)\t", t1)

    print("\nAll variables evaluated against 5 equations for motion in 1 direction.")
    print("\nSolutions needed: ", solutionsNeeded)
    print("\nResults: \n")
    
    
def removeUnitVariables(v = []):
    
    counts = 0

    try:
        v.remove(s)

    except:
        pass
    try:
        v.remove(m)
            
    except:
        pass
    
    return({'variablesToSolveFor': v, "count" : len(v)})

In [4]:
def calculateParticleMovement(_v = None, _v0 = None, _a = None,
                    _x = None, _x0 = None, _t0 = None, _t1 = None,
                             solutionsNeeded = [],
                             countOffreeVariablesAllowedInSolution = None,
                             createSummary = False):
    
    
    # ACCELERATION???
    # Create variables
    va, vi, v, v0, a, x, x0, t0, t1, m, s = sp.symbols('va, vi, v, v0, a, x, x0, t0, t1, m, s')
    
    # Provide summary
    if createSummary:
        createParticleMovementSummary(v0  = _v0, v = _v, a = _a, x = _x, x0 = _x0, t0 = _t0, t1 = _t1, solutionsNeeded = solutionsNeeded)
    
    
    # Physics Equations
    F1 = sp.Eq(x, x0 + v0 * t1 + .5 * a *t1**2)
    F2 = sp.Eq(v**2, v0**2 + 2 * a * (x - x0))
    F3 = sp.Eq(v, v0 + a * t1)
    F15 = sp.Eq(va, (x - x0) / (t1 - t0))
    
    F7 = sp.Eq(v, v0  + sp.integrate(a, (t1, 0, t1)))
    F8 = sp.Eq(x, x0  + sp.integrate(v, (t1, 0, t1)))
    F16 = sp.Eq(vi, sp.diff(_x, t1))
  
    
 
    # Create list of physics equations with values provided
    F4 = [i.subs({v:_v, v0: _v0, a:_a, x:_x, x0:_x0, t0:_t0, t1:_t1 }) for i in [F1, F2, F3, F7, F8, F15, F16]]
    # Create free symbols for each of the equations in F4
    F5 = [list(i.free_symbols) for i in F4]
    
    
    # Remove units before solutinos
    F12 = []
    for i in range(len(F5)):
        F12.append((removeUnitVariables(F5[i])))


    F6 = []
    # Get solutions for each of the equations for values other than unit variables
    for i in range(len(F4)):
        for j in range(len(F12[i]['variablesToSolveFor'])): 
            solutions = sp.solve(F4[i], F12[i]['variablesToSolveFor'][j])
            # Break all solutions into sgle solutions
            for k in range(len(solutions)):
                singleSolution = sp.Eq(F12[i]['variablesToSolveFor'][j], sp.solve(F4[i], F12[i]['variablesToSolveFor'][j])[k])
                
                # If no particular solutions sought, return everything
                if len(solutionsNeeded) == 0:
                    F6.append(singleSolution)
                # if only certain solutions are needed return these 
                elif singleSolution.lhs in solutionsNeeded:
                    F6.append(singleSolution)
                    
    F9 = [list(i.free_symbols) for i in F6]
    F11 = [removeUnitVariables(i) for i in F9]
    
    if countOffreeVariablesAllowedInSolution is not None:
        F13 = ([(idx) for idx, solution in enumerate(F11) if solution['count'] <= countOffreeVariablesAllowedInSolution])    
        F14 = np.array(F6)[F13]
        return(F14)
    else:
        return(F6)


In [5]:
P2 = calculateParticleMovement(_x = 20 * m + (5 * (m / s**2)) * t1**2,
                               solutionsNeeded=[vi],
                               createSummary=True)
renderListToLatex(P2)

###################### SUMMARY #######################
Initial Position 	 (x0):	 None
Final Position 		 (x):	 20*m + 5*m*t1**2/s**2
Initial Velocity 	 (v0):	 None
Final Velocity 		 (v):	 None
Acceleration 		 (a):	 None
Initial Time:		 (t0)	 None
Final Time:		 (t1)	 None

All variables evaluated against 5 equations for motion in 1 direction.

Solutions needed:  [vi]

Results: 



In [7]:
sp.diff(20 * m + (5 * (m / s**2) * t1**2), t1)

10*m*t1/s**2

In [77]:
# P2 = calculateParticleMovement(_a = 2 * (m / s**2) - (.1 * (m / s**3)) * t1,
#                               _v0 = 10 * (m / s),
#                               _x0 = 50 * m,
#                                solutionsNeeded=[x, va],
#                                 countOffreeVariablesAllowedInSolution = 1,
#                                createSummary=True)
#renderListToLatex(P2)

In [78]:
x0, x1, y0, y1, z0, z1, v0, v1, a0, a1, t, t0, t1, pFuX, pFuY, pFuZ, vFu, aFu  = sp.symbols('x0, x1, y0, y1, z0, z1, v0, v1, a0, a1, t, t0, t1, pFuX, pFuY, pFuZ, vFu, aFu ')

In [199]:
def createParticleMovementSummary(x0 = None, x1 = None, y0 = None, 
                                  y1 = None, z0 = None, z1 = None, 
                                  v0 = None, v1 = None,  a0 = None, 
                                  a1 = None, t0 = None,  t1 = None, 
                                  pFunX = None, pFunY = None, pFunZ = None, 
                                  vFu = None, aFu = None, solutionsNeeded = []):
    #create summary
    print("###################### SUMMARY #######################")
    print("Initial x Position \t (x0):\t", x0)
    print("Final x Position \t (x):\t", x1)
    print("Position function (i) \t (xf):\t", pFunX)
    print("Position function (j) \t (xf):\t", pFunY)
    print("Initial Velocity \t (v0):\t", v0)
    print("Final Velocity \t\t (v):\t", v1)
    print("Acceleration \t\t (a):\t", a0)
    print("Initial Time:\t\t (t0)\t", t0)
    print("Final Time:\t\t (t1)\t", t1)

    print("\nAll variables evaluated against 5 equations for motion in multiple directions.")
    print("\nSolutions needed: ", solutionsNeeded)
    
    print("###################### LAWS #######################\n\n")
    
    print("\nResults: \n")
    

def calculateParticleMovementMultipleDimension(_x0 = None, _x1 = None, _y0 = None, 
                                               _y1 = None, _z0 = None,_z1 = None, 
                                               _v0 = None, _v1 = None, _a0 = None, 
                                               _a1 = None, _t0 = None, _t1 = None, 
                                               _pFunX = None,_pFunY = None,_pFunZ = None, 
                                               _vFunX = None, _aFun = None, 
                             solutionsNeeded = [],
                             countOffreeVariablesAllowedInSolution = None,
                             createSummary = False):
    # Create variables
    i, j, k, x0, x1, y0, y1, z0, z1, v0, v1, a0, a1, t0, t1, pFunX, pFunY, pFunZ, vFunX, vFunY, vFunZ, aFu = sp.symbols("i, j, k, x0, x1, y0, y1, z0, z1, v0, v1, a0, a1, t0, t1, pFunX, pFunY, pFunZ, vFunX, vFunY, vFunZ, aFu") 
     
    
    if createSummary:
        createParticleMovementSummary(
            pFunX = _pFunX, 
            pFunY = _pFunY, 
            solutionsNeeded = solutionsNeeded)
    
    
    
### TIME 
    if _t0 == None:
        t0 = 0
        
    if _t1 != None:
        __t1 = _t1 - t0
        
 
    pFunX = _pFunX
    pFunY = _pFunY
    vFunX = sp.diff(pFunX, x1)
    vFunY = sp.diff(pFunY, y1)
    

    F1 = [i.subs({x0: _x0, y1:_y1, t1: __t1}) for i in [pFunX, pFunY, sp.Eq(z1, 0)]]
       
    F2 = [i.subs({x0: _x0, x1:_x1, t1: __t1}) for i in [pFunX, pFunY, sp.Eq(z1, 0)]]
    

    
    eucDistance = sp.sqrt(sum(([F1[i].rhs**2 for i in range(len(F1))])))
    
    
    return([F1, eucDistance])
    
    


In [200]:
xChoice =  sp.Eq(x1, 2 * m - (.25 * (m /  s**2)) * t1**2)
yChoice = sp.Eq(y1, 1 * (m / s) * t1 + .025 * (m / s**3) * t1**3)
#renderListToLatex([xChoice, yChoice])
calculateParticleMovementMultipleDimension(_pFunX=xChoice,
                                           _pFunY=yChoice,
                                           _t1 = 2 * s,
                                          createSummary=True)

###################### SUMMARY #######################
Initial x Position 	 (x0):	 None
Final x Position 	 (x):	 None
Position function (i) 	 (xf):	 Eq(x1, 2*m - 0.25*m*t1**2/s**2)
Position function (j) 	 (xf):	 Eq(y1, m*t1/s + 0.025*m*t1**3/s**3)
Initial Velocity 	 (v0):	 None
Final Velocity 		 (v):	 None
Acceleration 		 (a):	 None
Initial Time:		 (t0)	 None
Final Time:		 (t1)	 None

All variables evaluated against 5 equations for motion in multiple directions.

Solutions needed:  []
###################### LAWS #######################



Results: 



[[Eq(x1, 1.0*m), Eq(y1, 2.2*m), Eq(z1, 0)], 2.41660919471891*sqrt(m**2)]